In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

In [ ]:
!nvcc --version

In [13]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-kp3llobe
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-kp3llobe
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=cce05dec43bc507bef526f85898c357d05bc2e60774d60e1836bd66cdac72a89
  Stored in directory: /tmp/pip-ephem-wheel-cache-yboxfh5h/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [14]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


### Assignment A1:  Implement Parallel Reduction using Min, Max, Sum and Average operations. 

In [25]:
%%cu

        
#include<iostream>
#include<cstdio>
#include<cstdlib>
#include<cuda_runtime.h>
using namespace std;


__global__ void minimum(int *input)
{
	int tid=threadIdx.x;
	auto step_size=1;
  int number_of_threads=blockDim.x;
  
  while(number_of_threads>0)
  {
      if(tid<number_of_threads)
      {
          int first=tid*step_size*2;
          int second=first+step_size;
          if(input[second]<input[first])
            input[first]=input[second];
      }
      step_size=step_size*2;
      number_of_threads/=2;
  }

}

__global__ void max(int *input)
{
   int tid=threadIdx.x;
   auto step_size=1;
   int number_of_threads=blockDim.x;
   
   while(number_of_threads>0)
   {
       if(tid<number_of_threads)
       {
           int first=tid*step_size*2;
           int second=first+step_size;
           if(input[second]>input[first])
            input[first]=input[second];
       }
       step_size*=2;
       number_of_threads/=2;
   }
}

__global__ void sum(int *input)
{
    const int tid=threadIdx.x;
    auto  step_size=1;
    int number_of_threads=blockDim.x;
    while(number_of_threads>0)
    {
        if(tid<number_of_threads)
        {
            const int first=tid*step_size*2;
            const int second=first+step_size;
            input[first]=input[first]+input[second];
        }
    step_size = step_size*2;; 
		number_of_threads =number_of_threads/2;
    }
}

__global__ void average(int *input)
{
    const int tid=threadIdx.x;
    auto  step_size=1;
    int number_of_threads=blockDim.x;
    int totalElements=number_of_threads*2;
    while(number_of_threads>0)
    {
        if(tid<number_of_threads)
        {
            const int first=tid*step_size*2;
            const int second=first+step_size;
            input[first]=input[first]+input[second];
        }
        step_size = step_size*2;; 
		number_of_threads =number_of_threads/2;
    }
    input[0]=input[0]/totalElements;
}

int main()
{
	int n;
	n=100;
  cout<<"No of Elements: "<<n<<endl;
  srand(n);
	int *arr = new int[n];
	for(int i=0;i<n;i++)
	{
		arr[i]=rand()%20000;
	}

	int size=n*sizeof(int); //calculate no. of bytes for array
	int *arr_d,result1;
	
  //#max operation
  int *arr_max,maxValue;
  cudaMalloc(&arr_max,size);
	cudaMemcpy(arr_max,arr,size,cudaMemcpyHostToDevice);
  //#calling kernel max function
  max<<<1,n/2>>>(arr_max);
	cudaMemcpy(&maxValue,arr_max,sizeof(int),cudaMemcpyDeviceToHost);
	cout<<"Maximun: "<<maxValue<<endl;

  //#min Operation
	cudaMalloc(&arr_d,size);
	cudaMemcpy(arr_d,arr,size,cudaMemcpyHostToDevice);
  //#calling minimum kernel function
  minimum<<<1,n/2>>>(arr_d);
	cudaMemcpy(&result1,arr_d,sizeof(int),cudaMemcpyDeviceToHost);
	cout<<"Minimum: "<<result1<<endl;
    
  //#sum operation
  int *arr_sum,sumValue;
  cudaMalloc(&arr_sum,size);
	cudaMemcpy(arr_sum,arr,size,cudaMemcpyHostToDevice);
 //#calling kernal function
  sum<<<1,n/2>>>(arr_sum);
	cudaMemcpy(&sumValue,arr_sum,sizeof(int),cudaMemcpyDeviceToHost);
	cout<<"Sum: "<<sumValue<<endl; 
  //cout<<"Average:  "<<(sumValue/n)<<endl; 
   
  //#avg operation
  int *arr_avg,avgValue;
  cudaMalloc(&arr_avg,size);
	cudaMemcpy(arr_avg,arr,size,cudaMemcpyHostToDevice);
  //#calling kernal function
  average<<<1,n/2>>>(arr_avg);
	cudaMemcpy(&avgValue,arr_avg,sizeof(int),cudaMemcpyDeviceToHost);
	cout<<"Average: "<<avgValue<<endl; 
  
   
  //# Free all allcated device memeory
   cudaFree(arr_d);
   cudaFree(arr_sum);
   cudaFree(arr_max);
   cudaFree(arr_avg);
  
return 0;

}

No of Elements: 100
Maximun: 19936
Minimum: 180
Sum: 633114
Average: 6331

